In [2]:
import pandas as pd
from pathlib import Path

In [3]:
# puxa os dados direto da fonte
URL = "https://docs.google.com/spreadsheets/d/1mHfc3UE_vJ6vKhufAzYNsOpEPQW-GDPG6q5TchyO9Ms/export?format=csv"

In [32]:
df = pd.read_csv(URL, skiprows=1, usecols=range(1,26), decimal=',', thousands='.')

In [33]:
df = df[~df.Código.isna()].copy()

In [34]:
df.drop(columns=df.columns[15:], inplace=True)

In [42]:
df.dimensao_m

0       148.47
1        31.17
2        37.92
3        34.74
4        125.1
        ...   
260           
261    5065.45
262    5827.53
263    6682.63
264      200.0
Name: dimensao_m, Length: 265, dtype: object

In [35]:
def rewrite_columns(string):
    replacing = [(" ", "_"), ("ã", "a"), ("á", "a"), ("õ", "o"), ("²", ""), ("ç", "c"), ("é", "e"), ("í", "i"), ("ó", "o"), ("ú", "u"), ("(", ""), (")", ""), ("%", "")]
    for pair in replacing:
        string = string.replace(pair[0], pair[1])
    return string.lower()


df.columns = [rewrite_columns(x) for x in df.columns]

In [36]:
ambientes_origem = [x for x in df.columns if 'ambiente_origem' in x]

df["origem"] =  df[ambientes_origem].apply(lambda x: [ amb.split(' - ')[0] for amb in x.values if type(amb) == str ], axis=1)

df.drop(columns=ambientes_origem, inplace=True)

In [37]:
df.fillna("", inplace=True)

/tmp/ipykernel_843588/2065217620.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


In [38]:
dicionarios = df.to_dict('records')

In [39]:
registros = []

for item in dicionarios:
    key = item.pop('codigo')
    nome = item.pop('nome')
    origem = item.pop('origem')
    registros.append({
        'nome': nome,
        'origem': origem,
        'codigo': key,
        'detalhes':  item})

In [40]:
import json 

OUTPUT_PATH = Path("json0929") / "ambientes-norte.json"

with open(OUTPUT_PATH, "w") as fP:
    json.dump(registros, fP, indent=2, ensure_ascii=False)